In [ ]:
#-------------------------- Standard Imports --------------------------#
%reload_ext autoreload
%autoreload 2
import kdephys as kde
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import acr
import xarray as xr
from scipy import stats
# ---------------------------- EXTRAS --------------------------------#
from kdephys.plot.main import _title, bp_plot
import kdephys.utils.spectral as sp
bands = sp.bands

import warnings
warnings.filterwarnings('ignore')
import matplotlib as mpl
from acr.utils import *
from acr.utils import COND_ORDER
pu = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/pub_utils.py', 'pu')
dag = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/data_agg.py', 'dag')

In [ ]:
exp = 'swi'

In [ ]:
full_hyps = {}
hyp_dicts = {}
for subject in som_animals:
    full_hyps[subject] = acr.io.load_hypno_full_exp(subject, sub_swi_exps[subject][0])
    hyp_dicts[subject] = acr.hypnogram_utils.create_acr_hyp_dict(subject, sub_swi_exps[subject][0])
for subject in acr_animals:
    full_hyps[subject] = acr.io.load_hypno_full_exp(subject, sub_swi_exps[subject][0])
    hyp_dicts[subject] = acr.hypnogram_utils.create_acr_hyp_dict(subject, sub_swi_exps[subject][0])

In [ ]:
oodfs = []
for subject in som_animals:
    for exp in sub_swi_exps[subject]:
        print(subject, exp)
        wp = dag.load_base_oodf(subject, exp, dftype='whole-probe')
        wp = acr.oo_utils.label_oodf_with_states(wp, full_hyps[subject])
        wp = acr.oo_utils.label_oodf_full_bl(wp)
        wp = acr.oo_utils.label_oodf_hyp_dict_conditions(wp, hyp_dicts[subject])
        wp = acr.oo_utils.relativize_oodf(wp, ref_to_col='full_bl', ref_to_val='True', avg_method='mean', col_to_relativize='duration')
        oodfs.append(wp)
odf = pl.concat(oodfs)
offdf = odf.offs().filter(pl.col('condition') != 'None')
off_means = offdf.group_by(['subject', 'probe', 'condition']).agg(pl.col('duration_rel').mean())
ondf = odf.ons().filter(pl.col('condition') != 'None')
on_means = ondf.group_by(['subject', 'probe', 'condition']).agg(pl.col('duration_rel').mean())

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(30, 15))
sns.boxplot(data=on_means, x='condition', y='duration_rel', hue='probe', hue_order=['NNXr', 'NNXo'], palette=[NNXR_GRAY, SOM_BLUE], ax=ax[0], order=COND_ORDER)
sns.stripplot(data=on_means, x='condition', y='duration_rel', hue='probe', dodge=True, hue_order=['NNXr', 'NNXo'], palette=['red', 'black'], alpha=0.8, ax=ax[0], s=5, order=COND_ORDER)
ax[0].set_ylim(0, 2.5)
ax[0].axhline(y=1, color='red', linestyle='--')
ax[0].set_title('ON Period Duration by Subject, Relative to Full Baseline')

sns.boxplot(data=off_means, x='condition', y='duration_rel', hue='probe', hue_order=['NNXr', 'NNXo'], palette=[NNXR_GRAY, SOM_BLUE], ax=ax[1], order=COND_ORDER)
sns.stripplot(data=off_means, x='condition', y='duration_rel', hue='probe', dodge=True, hue_order=['NNXr', 'NNXo'], palette=['red', 'black'], alpha=0.8, ax=ax[1], s=5, order=COND_ORDER)
ax[1].set_ylim(0, 2.5)
ax[1].axhline(y=1, color='red', linestyle='--')
ax[1].set_title('OFF Period Duration by Subject, Relative to Full Baseline')

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(30, 15))
sns.boxplot(data=ondf, x='condition', y='duration', hue='probe', hue_order=['NNXr', 'NNXo'], palette=[NNXR_GRAY, SOM_BLUE], ax=ax[0], order=COND_ORDER)
ax[0].set_title('ON Period Duration by Subject, Raw Values')
ax[0].set_ylim(0, 2)

sns.boxplot(data=offdf, x='condition', y='duration', hue='probe', hue_order=['NNXr', 'NNXo'], palette=[NNXR_GRAY, SOM_BLUE], ax=ax[1], order=COND_ORDER)
ax[1].set_title('OFF Period Duration by Subject, Raw Values')

# Frequency

In [ ]:
freq_dfs = []
for subject in offdf['subject'].unique():
    subdf = odf.offs().filter(pl.col('subject')==subject)
    subfreq = acr.oo_utils.calc_off_freq(subdf, window='5s')
    subfreq = acr.oo_utils.label_oodf_with_states(subfreq, full_hyps[subject])
    subfreq = acr.oo_utils.label_oodf_full_bl(subfreq)
    subfreq = acr.oo_utils.label_oodf_hyp_dict_conditions(subfreq, hyp_dicts[subject])
    subfreq = acr.oo_utils.relativize_oodf(subfreq, ref_to_col='full_bl', ref_to_val='True', avg_method='mean', col_to_relativize='off_freq_per_s')
    freq_dfs.append(subfreq)
fdf = pl.concat(freq_dfs)
fdf = fdf.filter(pl.col('condition')!='None')
freq_means = fdf.group_by(['subject', 'probe', 'condition']).agg(pl.col('off_freq_per_s_rel').mean())

In [ ]:
f, ax = plt.subplots(figsize=(25, 10))
sns.boxplot(data=freq_means, x='condition', y='off_freq_per_s_rel', hue='probe', hue_order=['NNXr', 'NNXo'], palette=[NNXR_GRAY, SOM_BLUE], ax=ax, order=COND_ORDER)
sns.stripplot(data=freq_means, x='condition', y='off_freq_per_s_rel', hue='probe', dodge=True, hue_order=['NNXr', 'NNXo'], palette=['red', 'black'], alpha=0.8, ax=ax, s=5, order=COND_ORDER)
ax.axhline(y=1, color='red', linestyle='--')

ax.set_title('OFF Period Frequency by Subject, Relative to Full Baseline')


# ACR_SUBJECTS

In [ ]:
oodfs = []
for subject in acr_animals:
    for exp in sub_swi_exps[subject]:
        print(subject, exp)
        wp = dag.load_base_oodf(subject, exp, dftype='whole-probe')
        wp = acr.oo_utils.label_oodf_with_states(wp, full_hyps[subject])
        wp = acr.oo_utils.label_oodf_full_bl(wp)
        wp = acr.oo_utils.label_oodf_hyp_dict_conditions(wp, hyp_dicts[subject])
        wp = acr.oo_utils.relativize_oodf(wp, ref_to_col='full_bl', ref_to_val='True', avg_method='mean', col_to_relativize='duration')
        oodfs.append(wp)
odf = pl.concat(oodfs)
offdf = odf.offs().filter(pl.col('condition') != 'None')
off_means = offdf.group_by(['subject', 'probe', 'condition']).agg(pl.col('duration_rel').mean())
ondf = odf.ons().filter(pl.col('condition') != 'None')
on_means = ondf.group_by(['subject', 'probe', 'condition']).agg(pl.col('duration_rel').mean())

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(30, 15))
sns.boxplot(data=on_means, x='condition', y='duration_rel', hue='probe', hue_order=['NNXr', 'NNXo'], palette=[NNXR_GRAY, ACR_BLUE], ax=ax[0], order=COND_ORDER)
sns.stripplot(data=on_means, x='condition', y='duration_rel', hue='probe', dodge=True, hue_order=['NNXr', 'NNXo'], palette=['red', 'black'], alpha=0.8, ax=ax[0], s=5, order=COND_ORDER)
ax[0].set_ylim(0, 2.5)
ax[0].axhline(y=1, color='red', linestyle='--')
ax[0].set_title('ON Period Duration by Subject, Relative to Full Baseline')

sns.boxplot(data=off_means, x='condition', y='duration_rel', hue='probe', hue_order=['NNXr', 'NNXo'], palette=[NNXR_GRAY, ACR_BLUE], ax=ax[1], order=COND_ORDER)
sns.stripplot(data=off_means, x='condition', y='duration_rel', hue='probe', dodge=True, hue_order=['NNXr', 'NNXo'], palette=['red', 'black'], alpha=0.8, ax=ax[1], s=5, order=COND_ORDER)
ax[1].set_ylim(0, 2.5)
ax[1].axhline(y=1, color='red', linestyle='--')
ax[1].set_title('OFF Period Duration by Subject, Relative to Full Baseline')

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(30, 15))
sns.boxplot(data=ondf, x='condition', y='duration', hue='probe', hue_order=['NNXr', 'NNXo'], palette=[NNXR_GRAY, ACR_BLUE], ax=ax[0], order=COND_ORDER)
ax[0].set_title('ON Period Duration by Subject, Raw Values')
ax[0].set_ylim(0, 2)

sns.boxplot(data=offdf, x='condition', y='duration', hue='probe', hue_order=['NNXr', 'NNXo'], palette=[NNXR_GRAY, ACR_BLUE], ax=ax[1], order=COND_ORDER)
ax[1].set_title('OFF Period Duration by Subject, Raw Values')

## Frequency

In [ ]:
freq_dfs = []
for subject in offdf['subject'].unique():
    subdf = odf.offs().filter(pl.col('subject')==subject)
    subfreq = acr.oo_utils.calc_off_freq(subdf, window='5s')
    subfreq = acr.oo_utils.label_oodf_with_states(subfreq, full_hyps[subject])
    subfreq = acr.oo_utils.label_oodf_full_bl(subfreq)
    subfreq = acr.oo_utils.label_oodf_hyp_dict_conditions(subfreq, hyp_dicts[subject])
    subfreq = acr.oo_utils.relativize_oodf(subfreq, ref_to_col='full_bl', ref_to_val='True', avg_method='mean', col_to_relativize='off_freq_per_s')
    freq_dfs.append(subfreq)
fdf = pl.concat(freq_dfs)
fdf = fdf.filter(pl.col('condition')!='None')
freq_means = fdf.group_by(['subject', 'probe', 'condition']).agg(pl.col('off_freq_per_s_rel').mean())

In [ ]:
f, ax = plt.subplots(figsize=(25, 10))
sns.boxplot(data=freq_means, x='condition', y='off_freq_per_s_rel', hue='probe', hue_order=['NNXr', 'NNXo'], palette=[NNXR_GRAY, ACR_BLUE], ax=ax, order=COND_ORDER)
sns.stripplot(data=freq_means, x='condition', y='off_freq_per_s_rel', hue='probe', dodge=True, hue_order=['NNXr', 'NNXo'], palette=['red', 'black'], alpha=0.8, ax=ax, s=5, order=COND_ORDER)
ax.axhline(y=1, color='red', linestyle='--')

ax.set_title('OFF Period Frequency by Subject, Relative to Full Baseline')
